In [1]:
import os

import pandas as pd
pd.options.display.max_colwidth=200

from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

import warnings
warnings.filterwarnings('ignore')

import re
import numpy as np

from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

In [ ]:
#stops

In [2]:
train = pd.read_csv('train_E6oV3lV.csv')
print train.shape

(31962, 3)


In [42]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run
1,2,0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in urð±!!! ðððð ð¦ð¦ð¦
4,5,0,factsguide: society now #motivation


In [43]:
train[train.label==1].head()

,id,label,tweet
13,14,1,@user #cnn calls #michigan middle school 'build the wall' chant '' #tcot
14,15,1,no comment! in #australia #opkillingbay #seashepherd #helpcovedolphins #thecove #helpcovedolphins
17,18,1,retweet if you agree!
23,24,1,@user @user lumpy says i am a . prove it lumpy.
34,35,1,it's unbelievable that in the 21st century we'd need something like this. again. #neverump #xenophobia


In [44]:
## Postive VS Negative Tweets
print train.label.value_counts()
print '% of hate tweets among all tweets',100*2242.0/train.shape[0]

0    29720
1     2242
dtype: int64
% of hate tweets among all tweets 7.01457981353


In [45]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train["tweet"], train["label"], test_size=0.3, random_state=4242)

train_set = pd.concat([x_train,y_train],axis=1)
test_set = pd.concat([x_test,y_test],axis=1)
train_set.columns = ["tweet","label"]
test_set.columns = ["tweet","label"]
print train_set.shape
print test_set.shape

(22373, 2)
(9589, 2)


In [47]:
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

remove_words = ['user',"frm", "u", "urs", "n", "ur", "b", "mro", "mo", "tmr", "k", "ok",
"lol", "haha", "w", "moro", "yah", "cya", "cu", "eh", "hm", "hmm",
"yall", "xoxo", "yolo", "em", "v", "ver", "hav", "vry", "shud", "wer",
"abt", "bc", "wen", "jus", "tht", "fr", "hs", "r", "wud", "cud"]

def PreProcess(tweet): 
    #tweet = str(tweet)
    tweet = tweet.lower() ## Convert the text to lower case
    tweet = BeautifulSoup(tweet, "lxml").get_text() ## extract only text by dropping URLS
    tweet=re.sub('[^0-9a-zA-Z]', ' ', tweet) ## extract only alpha numeric from a text
    #tweet = re.sub("[^a-zA-Z]", " ", tweet)
    tweet = ' '.join([word.strip() for word in tweet.split() if word not in stops]) ## Remove eng stop words
    tweet = ' '.join([word for word in tweet.split() if word not in remove_words]) ## Remove specific words
    return tweet

In [10]:
#stops

In [48]:
train_set['cleaned_tweet'] = train_set.tweet.apply(PreProcess)
test_set['cleaned_tweet'] = test_set.tweet.apply(PreProcess)

In [49]:
train_set.head()

,tweet,label,cleaned_tweet
31471,just watched this with my son he saw it for the first time #ripbillybob #restinpeace #actors #movies #tosoon,0,watched son saw first time ripbillybob restinpeace actors movies tosoon
21663,just want to say i miss my little boy king every day but today i was missing one son today #king,0,want say miss little boy king every day today missing one son today king
11277,when you are a pa of this incredible new company before it even launches!! pm me for details! ð #maelle,0,pa incredible new company even launches pm details maelle
23305,smooth new year's eve! marijuana unleashed at ibooks.,1,smooth new year eve marijuana unleashed ibooks
17551,hardcore music nude asian women free,0,hardcore music nude asian women free


In [50]:
### Building TFIDF features### 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1, 2),analyzer='word')


train_tfidf = vectorizer.fit_transform(train_set.cleaned_tweet)

test_tfidf = vectorizer.transform(test_set.cleaned_tweet)


In [15]:
#vectorizer.get_feature_names()

In [110]:
#help(TfidfVectorizer)

In [51]:
#train_tfidf = train_tfidf[4::]
print train_tfidf.shape
print test_tfidf.shape

(22373, 144365)
(9589, 144365)


In [52]:
from sklearn.decomposition import TruncatedSVD
n_comp = 10
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(train_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]

In [53]:
## get the list of features 
import xgboost as xgb
d_train = xgb.DMatrix(train_svd, label=train_set.label)

params = {}
params['objective'] = 'binary:logistic'
#params['objective'] = 'multi:softmax'
#params['objective'] = 'multi:softprob'
#params['eval_metric'] = 'merror'
#params['booster']="gblinear"
params['eta'] = 0.1
params['max_depth'] = 10
params["subsample"] = 0.5
#params['num_class'] = 2 # Three Classes
params["min_child_weight"] = 1
params["colsample_bytree"] = 0.5

model = xgb.cv(dtrain=d_train,nfold=3,params=params,num_boost_round=20)
model = xgb.train(dtrain=d_train,params=params,num_boost_round=10)

[0]	cv-test-error:0.068929+0.001159	cv-train-error:0.068504+0.001118
[1]	cv-test-error:0.069152+0.001479	cv-train-error:0.069107+0.000798
[2]	cv-test-error:0.068482+0.001283	cv-train-error:0.068481+0.001174
[3]	cv-test-error:0.068705+0.001596	cv-train-error:0.068548+0.001056
[4]	cv-test-error:0.068705+0.001596	cv-train-error:0.068571+0.001140
[5]	cv-test-error:0.068258+0.001505	cv-train-error:0.068236+0.000829
[6]	cv-test-error:0.068214+0.001554	cv-train-error:0.068236+0.000779
[7]	cv-test-error:0.068571+0.001972	cv-train-error:0.068370+0.000259
[8]	cv-test-error:0.068526+0.001829	cv-train-error:0.068415+0.000662
[9]	cv-test-error:0.068615+0.001917	cv-train-error:0.068392+0.000289
[10]	cv-test-error:0.068258+0.002046	cv-train-error:0.067945+0.000669
[11]	cv-test-error:0.068347+0.001987	cv-train-error:0.068169+0.000439
[12]	cv-test-error:0.067945+0.001644	cv-train-error:0.067923+0.000522
[13]	cv-test-error:0.067856+0.001653	cv-train-error:0.067699+0.000743
[14]	cv-test-error:0.067632+0.

In [56]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

test_svd_tf = xgb.DMatrix(test_svd)
xgb_pred=model.predict(test_svd_tf)
xgb_pred_class = np.where(xgb_pred>0.3,1,0)
print accuracy_score(y_true=test_set.label, y_pred=xgb_pred_class)
print f1_score(y_true=test_set.label, y_pred=xgb_pred_class, average='macro')

0.93534258004
0.148351648352


In [57]:
from sklearn.svm import SVC, LinearSVC, NuSVC

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

clf = LinearSVC(C=4)
clf.fit(train_tfidf,train_set.label)
svm_pred_test_label=clf.predict(test_tfidf)

print accuracy_score(y_true=test_set.label, y_pred=svm_pred_test_label)
print f1_score(y_true=test_set.label, y_pred=svm_pred_test_label, average='macro')


0.966106997601
0.717636837533


In [58]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.009)
clf.fit(train_tfidf,train_set.label)
svm_pred_test_label=clf.predict(test_tfidf)

print accuracy_score(y_true=test_set.label, y_pred=svm_pred_test_label)
print f1_score(y_true=test_set.label, y_pred=svm_pred_test_label, average='macro')


0.964438419022
0.688014638609
